# How to spinfoam amplitudes
## Contracting the EPRL vertex amplitudes

In this notebook we contract the EPRL vertices (previously computed) in order to obtain the Δ4 amplitude.

Since the required vertices have already been computed, 
the cells below can be compiled just installing the required packages, even if the sl2cfoam-next is not installed. 

In [1]:
# required pkgs  
using HalfIntegers
using JLD2   
using DelimitedFiles
using DataFrames

In [2]:
# Barbero-Immirzi parameter
γ = 1.2

# current folder
root_dir = pwd()

# homogeneous cut-off 
Δl = 0;

Let's start by showing a way to contract amplitudes.

Notice that in julia best performance is achieved by using a function

In [133]:
# value of all boundary spins (symmetric configuration)    
j = 1.0

# value of all boundary intertwiners (symmetric configuration)   
i_boundary = 0

# julia arrays start at index 1   
i_b = i_boundary + 1    
    
# folder with stored vertex amplitudes 
vertex_path = "$(root_dir)/vertex_ampls/Immirzi_$(γ)/j_$(j)/Dl_$(Δl)"                     
                   
Δ4_amp = 0.0 
    
# if all boundary spins are integers and equal to j, bulk spin has range [0, 3j]   
j_bulk_min, j_bulk_max = 0, 3j 
    
for j_bulk = j_bulk_min:j_bulk_max

fulltensor_to_load = "$(vertex_path)/j_bulk_$(j_bulk)_fulltensor.jld2"
if(!isfile(fulltensor_to_load)) error("vertex fulltensor for γ=$(γ) j=$(j) Δl=$(Δl) not found, it must be computed first") end   
@load "$(fulltensor_to_load)" vertex 
                    
# ranges of intertwiner spaces touched by the bulk spin      
D = size(vertex[i_b,:,:,i_b,i_b])[1] # also [2] has the same range          
                    
Δ4_partial_amp = 0.0
  
for i_1234 in 1:D, i_1235 in 1:D, i_1236 in 1:D, i_1237 in 1:D
@inbounds Δ4_partial_amp += vertex[i_b,i_1234,i_1235,i_b,i_b]*vertex[i_b,i_1235,i_1236,i_b,i_b]*
                            vertex[i_b,i_1236,i_1237,i_b,i_b]*vertex[i_b,i_1237,i_1234,i_b,i_b]     
end     

# dimension of bulk spin     
Δ4_partial_amp *= (2j_bulk + 1)
            
Δ4_amp += Δ4_partial_amp  
                    
end # j_bulk cycle
               
@show Δ4_amp

Δ4_amp = 7.768396061116365e-39


7.768396061116365e-39

For convenience, we write a function that contracts all the required vertices for given lists of boundary parameters in the symmetric configuration.

We collect all the amplitudes in a matrix for increasing values of the cut-off Δl and different intertwiners configurations, 
then we store in CSV format.

In [134]:
function Δ4_assemble(Immirzi_list, j_boundary_list, Δl_max)
    
for γ in Immirzi_list

   for j in j_boundary_list    
        
      # if all boundary spins are equal to j, each boundary intertwiner has a range [0, 2j]     
      i_boundary_min, i_boundary_max = 0, convert(Int, 2j)
        
      # matrix in which the computed amplitudes will be stored         
      Δ4_amplitudes = zeros(Δl_max + 1, i_boundary_max + 1)           
      column_labes = String[]      

      for i_boundary = i_boundary_min:i_boundary_max
          
         i_b = i_boundary + 1 
         push!(column_labes, "i=$(i_boundary)")       
    
         for Δl = 0:Δl_max

            vertex_path = "$(root_dir)/vertex_ampls/Immirzi_$(γ)/j_$(j)/Dl_$(Δl)"                           
            Δ4_amp = 0.0 
                
            if (iszero(j%1)) j_bulk_min = 0 else j_bulk_min = 0.5 end       
            j_bulk_max = 3j
    
            for j_bulk = j_bulk_min:j_bulk_max

            fulltensor_to_load = "$(vertex_path)/j_bulk_$(j_bulk)_fulltensor.jld2"
            if(!isfile(fulltensor_to_load)) error("vertex fulltensor for γ=$(γ) j=$(j) Δl=$(Δl) not found, it must be computed first") end    
            @load "$(fulltensor_to_load)" vertex 
                        
               D = size(vertex[i_b,:,:,i_b,i_b])[1]                              
               Δ4_partial_amp = 0.0
  
                  for i_1234 in 1:D, i_1235 in 1:D, i_1236 in 1:D, i_1237 in 1:D
                  @inbounds Δ4_partial_amp += vertex[i_b,i_1234,i_1235,i_b,i_b]*vertex[i_b,i_1235,i_1236,i_b,i_b]*
                                              vertex[i_b,i_1236,i_1237,i_b,i_b]*vertex[i_b,i_1237,i_1234,i_b,i_b]     
                  end     
   
                  Δ4_partial_amp *= (2j_bulk + 1)            
                  Δ4_amp += Δ4_partial_amp 
                        
                  end # j_bulk cycle
                
               Δ4_amplitudes[Δl+1,i_b] = Δ4_amp       
        
               # storing the Δ4 amplitude in JLD2 format    
               Δ4_JLD2_path = "$(root_dir)/Delta_4_ampls/Immirzi_$(γ)/j_$(j)/JLD2_format/i_$(i_boundary)/"
               mkpath(Δ4_JLD2_path)         
               @save "$(Δ4_JLD2_path)/Dl_$(Δl).jld2" Δ4_amp
        
               end # cut-off cycle    
        
            end # i_boundary cycle    
                  
         # storing the Δ4 amplitudes in CSV format
         Δ4_CSV_path = "$(root_dir)/Delta_4_ampls/Immirzi_$(γ)/j_$(j)/CSV_format" 
         mkpath(Δ4_CSV_path)
         open("$(Δ4_CSV_path)/Delta_4_ampls_Dl_max_$(Δl_max).csv", "w") do io
         writedlm(io, Δ4_amplitudes, ',')
         end   
          
         # notice that i-th row is corresponds to Δl=i-1 
         Amplitudes = DataFrame(Δ4_amplitudes, column_labes)    
         println("Δ4 amplitudes for j = $(j) and γ = $(γ) are:\n", Amplitudes, "\n\n")

   end # j_boundary cycle       

end # Immirzi cycle   
    
end # end of function

Δ4_assemble (generic function with 1 method)

In [135]:
# list of Immirzi parameters 
Immirzi_list = [0.1, 1.2]

# list of boundary spins (symmetric configurations)
j_boundary_list = [0.5,1]

# maximal cut-off  
Δl_max = 15;

In [136]:
Δ4_assemble(Immirzi_list, j_boundary_list, Δl_max)

Δ4 amplitudes for j = 0.5 and γ = 0.1 are:
16×2 DataFrame
 Row │ i=0          i=1         
     │ Float64      Float64     
─────┼──────────────────────────
   1 │ 4.81247e-21  3.8768e-20
   2 │ 2.50346e-20  2.0026e-19
   3 │ 5.12196e-20  4.07089e-19
   4 │ 7.37273e-20  5.83916e-19
   5 │ 9.12362e-20  7.20837e-19
   6 │ 1.04391e-19  8.23385e-19
   7 │ 1.14328e-19  9.00632e-19
   8 │ 1.21897e-19  9.59364e-19
   9 │ 1.27762e-19  1.00479e-18
  10 │ 1.32368e-19  1.04041e-18
  11 │ 1.36043e-19  1.06881e-18
  12 │ 1.39013e-19  1.09173e-18
  13 │ 1.41444e-19  1.11047e-18
  14 │ 1.43456e-19  1.12598e-18
  15 │ 1.45138e-19  1.13894e-18
  16 │ 1.46559e-19  1.14987e-18


Δ4 amplitudes for j = 1.0 and γ = 0.1 are:
16×3 DataFrame
 Row │ i=0          i=1          i=2         
     │ Float64      Float64      Float64     
─────┼───────────────────────────────────────
   1 │ 8.11368e-27  5.95039e-26  1.73228e-25
   2 │ 3.87335e-26  2.76091e-25  8.4097e-25
   3 │ 7.74983e-26  5.21764e-25  1.64612e-24
 